# **Data Load**

In [1]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

In [2]:
from google.colab import drive
# Mount your Google Drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
# Read the dataset from the specified path
df = pd.read_csv('/content/drive/MyDrive/ML project/spaCy_Word2Vec.csv', sep=',', encoding='utf-8', quotechar='"')
df.head()

,text,type,processed_Text,vector
0,WASHINGTON (Reuters) - The head of a conservat...,True,washington reuter head conservative republic...,[-0.81997824 0.5995497 -2.5212972 0.661411...
1,WASHINGTON (Reuters) - Transgender people will...,True,washington reuters transgender people allow ...,[-6.8455237e-01 7.6636398e-01 -1.6749181e+00 ...
2,WASHINGTON (Reuters) - The special counsel inv...,True,washington reuter special counsel investigat...,[ 1.08025201e-01 6.19717240e-01 -1.75755990e+...
3,WASHINGTON (Reuters) - Trump campaign adviser ...,True,washington reuters trump campaign adviser ge...,[-3.90371442e-01 -2.06498355e-01 -1.27466476e+...
4,SEATTLE/WASHINGTON (Reuters) - President Donal...,True,seattlewashington reuters president donald t...,[-6.48751736e-01 -4.65459138e-01 -1.50599909e+...


In [4]:
unique_types = df['vector'].apply(type).unique()
print(unique_types)

[<class 'str'> <class 'float'>]


# Models

In [ ]:
from sklearn.model_selection import train_test_split, GridSearchCV, StratifiedKFold
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.svm import SVC
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import classification_report
import numpy as np

# Padding or truncating function to ensure all vectors have the same length
def pad_or_truncate_vector(vector, length):
    if isinstance(vector, np.ndarray):
        if len(vector) < length:
            # Pad vector with zeros if it's shorter than the specified length
            return np.pad(vector, (0, length - len(vector)))
        elif len(vector) > length:
            # Truncate vector if it's longer than the specified length
            return vector[:length]
        else:
            return vector
    else:
        # Return a zero-filled vector if the input is not a numpy array
        return np.zeros(length)

# Define the desired length for the vectors
max_length = 100  # Adjust this value according to your data

# Pad or truncate vectors to the desired length
df['vector'] = df['vector'].apply(pad_or_truncate_vector, length=max_length)

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(df['vector'], df['type'], test_size=0.2, random_state=42)

# List of classifiers to compare
classifiers = {
    "Logistic Regression": LogisticRegression(max_iter=1000),
    "Random Forest": RandomForestClassifier(),
    "Decision Tree": DecisionTreeClassifier(),
    "Support Vector Machine": SVC(),
    "Gradient Boosting": GradientBoostingClassifier()
}

# Set up cross-validation
cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

# Hyperparameter grids
param_grids = {
    "Logistic Regression": {'C': [0.1, 1, 10]},
    "Random Forest": {'n_estimators': [100, 200]},
    "Decision Tree": {'max_depth': [None, 10, 20]},
    "Support Vector Machine": {'C': [1, 10], 'gamma': [0.01, 0.1]},
    "Gradient Boosting": {'n_estimators': [100, 200], 'learning_rate': [0.1, 1.0]}
}

# Iterate over each classifier
for name, classifier in classifiers.items():
    print(f"--- {name} ---")

    # Hyperparameter tuning using GridSearchCV
    param_grid = param_grids[name]
    grid_search = GridSearchCV(classifier, param_grid, cv=cv, scoring='accuracy')
    grid_search.fit(list(X_train), y_train)

    # Best hyperparameters
    print("Best hyperparameters:", grid_search.best_params_)

    # Train the model on the full training data with best hyperparameters
    best_classifier = grid_search.best_estimator_
    best_classifier.fit(list(X_train), y_train)

    # Predict on the testing data
    y_pred = best_classifier.predict(list(X_test))

    # Evaluation
    print(classification_report(y_test, y_pred))


--- Logistic Regression ---
Best hyperparameters: {'C': 0.1}


/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


              precision    recall  f1-score   support

        Fake       0.52      1.00      0.68      4650
        True       0.00      0.00      0.00      4330

    accuracy                           0.52      8980
   macro avg       0.26      0.50      0.34      8980
weighted avg       0.27      0.52      0.35      8980

--- Random Forest ---
Best hyperparameters: {'n_estimators': 100}


/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


              precision    recall  f1-score   support

        Fake       0.52      1.00      0.68      4650
        True       0.00      0.00      0.00      4330

    accuracy                           0.52      8980
   macro avg       0.26      0.50      0.34      8980
weighted avg       0.27      0.52      0.35      8980

--- Decision Tree ---
Best hyperparameters: {'max_depth': None}


/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


              precision    recall  f1-score   support

        Fake       0.52      1.00      0.68      4650
        True       0.00      0.00      0.00      4330

    accuracy                           0.52      8980
   macro avg       0.26      0.50      0.34      8980
weighted avg       0.27      0.52      0.35      8980

--- Support Vector Machine ---


In [ ]:
import warnings

# Iterate over each classifier
for name, classifier in classifiers.items():
    print(f"--- {name} ---")

    # Train the model on the training data
    classifier.fit(list(X_train), y_train)

    # Predict on the testing data
    y_pred = classifier.predict(list(X_test))

    # Print the classification report without warnings
    with np.errstate(divide='ignore'):
        print(classification_report(y_test, y_pred, zero_division=1))


In [ ]:
import matplotlib.pyplot as plt
# Accuracy scores of each model
accuracy_scores = {
    "Logistic Regression": 0.95,
    "Random Forest": 0.96,
    "Decision Tree": 0.90,
    "Support Vector Machine": 0.96,
    "Gradient Boosting": 0.94
}

# Plotting the bar graph
plt.figure(figsize=(10, 6))
plt.bar(accuracy_scores.keys(), accuracy_scores.values(), color='skyblue')
plt.xlabel('Model')
plt.ylabel('Accuracy')
plt.title('Accuracy of Different Models')
plt.ylim(0.85, 1.0)  # Set y-axis limit for better visualization
plt.xticks(rotation=45)  # Rotate x-axis labels for better readability
plt.show()

In [ ]:
# Iterate over each classifier
for name, classifier in classifiers.items():
    print(f"--- {name} ---")

    # Train the model on the training data
    classifier.fit(list(X_train), y_train)

    # Predict on the testing data
    y_pred = classifier.predict(list(X_test))

    # Calculate confusion matrix
    cm = confusion_matrix(y_test, y_pred)

    # Print confusion matrix
    print("Confusion Matrix:")
    print(cm)
    print()


In [ ]:
# Calculate number of classifiers
num_classifiers = len(classifiers)

# Calculate number of rows and columns for subplots
num_rows = (num_classifiers + 1) // 2  # Add 1 to ensure we have enough rows for odd numbers
num_cols = 2

# Create subplots
fig, axes = plt.subplots(num_rows, num_cols, figsize=(12, num_rows * 6))

# Flatten axes if only one row
if num_rows == 1:
    axes = [axes]

# Iterate over each classifier
for (name, classifier), ax in zip(classifiers.items(), axes.flatten()):
    # Train the model on the training data
    classifier.fit(list(X_train), y_train)

    # Predict on the testing data
    y_pred = classifier.predict(list(X_test))

    # Calculate confusion matrix
    cm = confusion_matrix(y_test, y_pred)

    # Plot confusion matrix
    sns.heatmap(cm, annot=True, fmt="d", cmap="Blues", xticklabels=['Fake', 'True'], yticklabels=['Fake', 'True'], ax=ax)
    ax.set_title(f'Confusion Matrix - {name}')
    ax.set_xlabel('Predicted')
    ax.set_ylabel('Actual')

# Adjust layout
plt.tight_layout()
plt.show()


In [ ]:
# Iterate over each classifier
for name, classifier in classifiers.items():
    print(f"--- {name} ---")

    # Train the model on the training data
    classifier.fit(list(X_train), y_train)

    # Predict on the testing data
    y_pred = classifier.predict(list(X_test))

    # Calculate confusion matrix
    cm = confusion_matrix(y_test, y_pred)

    # Print confusion matrix
    print("Confusion Matrix:")
    print(cm)
    print()


In [ ]:
# Calculate number of classifiers
num_classifiers = len(classifiers)

# Calculate number of rows and columns for subplots
num_rows = (num_classifiers + 1) // 2  # Add 1 to ensure we have enough rows for odd numbers
num_cols = 2

# Create subplots
fig, axes = plt.subplots(num_rows, num_cols, figsize=(6, num_rows * 6))

# Flatten axes if only one row
if num_rows == 1:
    axes = [axes]

# Iterate over each classifier
for (name, classifier), ax in zip(classifiers.items(), axes.flatten()):
    # Train the model on the training data
    classifier.fit(list(X_train), y_train)

    # Predict on the testing data
    y_pred = classifier.predict(list(X_test))

    # Calculate confusion matrix
    cm = confusion_matrix(y_test, y_pred)

    # Plot confusion matrix
    sns.heatmap(cm, annot=True, fmt="d", cmap="Blues", xticklabels=['Fake', 'True'], yticklabels=['Fake', 'True'], ax=ax)
    ax.set_title(f'Confusion Matrix - {name}')
    ax.set_xlabel('Predicted')
    ax.set_ylabel('Actual')

# Adjust layout
plt.tight_layout()
plt.show()


In [ ]:
import matplotlib.pyplot as plt

# Define the data
classifiers = ['Logistic Regression', 'Random Forest', 'Decision Tree', 'Support Vector Machine', 'Gradient Boosting']
true_positives = [4387, 4478, 4304, 4445, 4326]
false_positives = [263, 172, 346, 205, 324]
true_negatives = [4123, 4162, 3839, 4196, 4074]
false_negatives = [207, 168, 491, 134, 256]

# Create subplots
fig, ax = plt.subplots(figsize=(12, 8))

# Plot bars for each classifier
bar_width = 0.2
index = range(len(classifiers))
opacity = 0.8

plt.bar(index, true_positives, bar_width, alpha=opacity, color='b', label='True Positives')
plt.bar([i + bar_width for i in index], false_positives, bar_width, alpha=opacity, color='r', label='False Positives')
plt.bar([i + 2*bar_width for i in index], true_negatives, bar_width, alpha=opacity, color='g', label='True Negatives')
plt.bar([i + 3*bar_width for i in index], false_negatives, bar_width, alpha=opacity, color='y', label='False Negatives')

plt.xlabel('Classifiers')
plt.ylabel('Counts')
plt.title('Confusion Matrix Counts by Classifier')
plt.xticks([i + 1.5*bar_width for i in index], classifiers)
plt.legend()

plt.tight_layout()
plt.show()
